# The Elements of Statistical Learning

Clasificación de puntos en un plano según dos posibles categorias, puntos rojos o azules

Este notebook incluye todos los pasos tipicos de un ejercicio de Machine Learning, enfocandose en un problema de clasificacion entre dos categorias (rojo o azul) de los puntos de un plano. Todo esto en busqueda del grafico 2.4 del libro The Elements. Este notebook se dividirá en las siguientes partes:

- Simulación de datos del train y del test
    - En muchos ejercicios de Machine Learning estos datos son dados como condicion inicial, pero en nuestro caso nosotros los simulamos. Debemos recordar que una de las suposiciones de ML es que los datos siguen cierta funcion de probabilidad y que en principio conociendo esta funcion podriamos simularlos
- Definicion de la función para calcular las predicciones o hacer la clasificacion (lineal,sigmoide, Knn, etc)
    - En el lenguaje de ML esta funcion es llamada Hipótesis
- Definicion de la función costo 
    -  En un primer caso usamos RSS como funcion de costo, pero en general podriamos considerar otras funciones de costo, dependiendo de las propiedades particulares de cada problema
- Minimización de la función costo para encontrar los parámetros libres de la hipótesis 
    - En un primer caso encontramos los valores de los parametros theta de la funcion lineal minimizando analiticamente la funcion costo RSS, en lenguaje de ML esta ecuación es denominada Normal Equation. 
    - Sin embargo es necesario notar que también existen otros metodos, que son numericos en general. Por ejemplo, anteriormente usamos la busqueda aleatoria de los paramteros seleccionando sistematicamente los valores que reducen RSS. Otro metodo, con nombre de hecho, es Gradient Descent que es el mas tipico de Machine Learning.
- Visualizacion de las predicciones en el train y el test
- Evaluacion cuantitativa de las predicciones en terminos de la accuracy y el test error
    - Estos resultados nos permitiran escoger los mejores modelos para el problema planteado
- Gráfico de la misclassification curves (Figure 2.4)

Inicialmente importamos la librería que utilizaremos:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

Ahora importamos la librería necesaria para la implementación de una red neuronal 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

## Simulación de datos:

Dado las indicaciones en la pagina 16 del libro "The Elements of Statistical Learning", se puede entender que el proceso de simulación del set de datos de entrenamiento es el siguiente.

1- Primero se escoge el color a simular

2- Dado el color, conocemos el centro y la matriz de covarianza de una gaussiana bivariada

3- Con esta gaussiana bivariada generamos 10 centros. Esto lo hacemos solo una vez porque nos permite generar las observaciones asociadas a un set fijo de gaussianas bivariadas, 10 en nuestro caso. La idea de la simulación es generar observaciones que representen instancias asociados a un set de gaussianas bivariadas, no con respecto a una sola de ellas

4- Con estos valores de los centros, mas una covarianza fija igual a la identidad/5, podemos asumir que conocemos los parametros de 10 gaussianas bivariadas

5- Considerando estas 10 gaussianas bivariadas podemos generar nuestras observaciones

6- Cada observacion se genera de la siguiente forma:

    1- Se escoge uno de los centros dados anteriormente y su gaussiana bivariada correspondiente
    2- Se genera un valor de la posicion siguiendo esta gaussiana bivariada   
    3- Se vuelve a 6.1 y se repite el proceso, 100 veces por cada color

### Definicion de los centros de las gaussianas bivariadas

In [ ]:
def centros_asociados_a_cada_color(color, kcentros):
   
    centro = np.array([0,0])
    covarianza = np.identity(2)
    
    if color == 0: #azul
        centro = np.array([1,0])
    
    if color == 1: #rojo
        centro = np.array([0,1])
        
    np.random.seed(15)   #esto fijará la semilla del random para sistematizar la aleatoriedad.
    lista_de_centros = np.random.multivariate_normal(centro,covarianza,kcentros).T
        
    return lista_de_centros

In [ ]:
centros_azul = centros_asociados_a_cada_color(0,10)
centros_rojo = centros_asociados_a_cada_color(1,10)

In [ ]:
def visualize_colors(ptos_azul, ptos_rojo):

    x1azul = ptos_azul[0]
    x2azul = ptos_azul[1]
    
    x1rojo = ptos_rojo[0]
    x2rojo = ptos_rojo[1]
    
    plt.plot(x1azul,x2azul,"bo")
    plt.plot(x1rojo,x2rojo,"ro")
    plt.axis([-4,4,-4,4])
    plt.title("Distribucion de puntos segun el color")
    plt.show() 

In [ ]:
visualize_colors(centros_azul, centros_rojo)

### Implementacion de la funcion que simula los datos

In [ ]:
def Simulacion(centros, N):
    
    Ncentros = centros.shape[1]
    
    #Inicializamos la lista de observaciones 
    observations = np.zeros((2,N))
    
    #Definimos un valor global para la covarianza
    covarianza = np.identity(2)/5
       
    #hacemos un loop entre 0 y N para generar todos las observaciones requeridas
    
    for obs in range(0,N):
        #generamos un numero aleatorio entre los números 0 y Ncentros-1 para escoger el centro de nuestra gaussiana bivariada
        random.seed(obs**2)
        indice_del_centro = random.randint(0, Ncentros-1)
        
        xcentro = centros[0,indice_del_centro]
        ycentro = centros[1,indice_del_centro]
        
        mk_centro = np.array([xcentro, ycentro])
        
        np.random.seed(obs+10)
        centro_aux = np.random.multivariate_normal(mk_centro,covarianza,1).T
    
        observations[0,obs] = centro_aux[0,0]
        observations[1,obs] = centro_aux[1,0]
    
    return observations

### Datos del conjunto de entrenamiento

In [ ]:
ptos_observados_azul_train =  Simulacion(centros_azul, 100)
ptos_observados_rojo_train =  Simulacion(centros_rojo, 100)

In [ ]:
visualize_colors(ptos_observados_azul_train, ptos_observados_rojo_train)

### Datos del  conjunto de testeo
Se generarán 5000 puntos para cada color (en total serán 10000 puntos) a partir de los mismos 20 centros usados para el conjunto de entrenamiento. 

In [ ]:
ptos_observados_azul_test =  Simulacion(centros_azul, 5000)
ptos_observados_rojo_test =  Simulacion(centros_rojo, 5000)

In [ ]:
visualize_colors(ptos_observados_azul_test, ptos_observados_rojo_test)

### Formación del Conjunto de Entrenamiento (SET_train) y del Conjunto de Testeo (SET_test) 

In [ ]:
SET_train=[]

x1azul_train = ptos_observados_azul_train[0,:]
x2azul_train = ptos_observados_azul_train[1,:]

x1rojo_train = ptos_observados_rojo_train[0,:]
x2rojo_train = ptos_observados_rojo_train[1,:]

# Se agregan estos puntos a SET_train con su correspondiente valor de Y. Si Y=0 corresponde a 
# la distribución normal azul. Si Y=1 corresponde a la distribución normal roja. Adicionalmente se agregará
# el bias al inicio de cada fila en SET_train.

for i in range(len(x1azul_train)): #Y=0
    SET_train.append([1,x1azul_train[i],x2azul_train[i],0])

for i in range(len(x1rojo_train)): #Y=1
    SET_train.append([1,x1rojo_train[i],x2rojo_train[i],1])

In [ ]:
SET_test=[]

x1azul_test = ptos_observados_azul_test[0,:]
x2azul_test = ptos_observados_azul_test[1,:]

x1rojo_test = ptos_observados_rojo_test[0,:]
x2rojo_test = ptos_observados_rojo_test[1,:]

for i in range(len(x1azul_test)): #Y=0
    SET_test.append([1,x1azul_test[i],x2azul_test[i],0])

for i in range(len(x1rojo_test)): #Y=1
    SET_test.append([1,x1rojo_test[i],x2rojo_test[i],1])

Ahora reordenamos de manera aleatoria cada uno de los SETs para no perder generalidades

In [ ]:
np.random.shuffle(SET_train)
np.random.shuffle(SET_test)

Los largos correspondientes a cada SET se muestran a continuación.

In [ ]:
print(len(SET_train), len(SET_test))

## Análisis de funciones para calcular predicciones.

## Caso 1: Mínimos cuadrados

- Hipótesis: función lineal
- Función costo: RSS 
- Minimización: theta obtenido teóricamente (Normal Equation)
- Evaluación: accuracy y test error

En este caso asumimos que el modelo de clasificación o hipotesis esta dado por una funcion lineal del tipo 

$$ h(x_1^{(i)},x_2^{(i)}) = \theta_0 + \theta_1 x_1^{(i)} + \theta_2 x_2^{(i)} $$

y que los valores de theta son obtenidos a partir de la busqueda del minimo de la funcion RSS con respecto a los datos del conjunto de entrenamiento. Este minimo puede ser encontrado analitcamente, como es nuestro caso actual, pero tambien puede ser encontrado numericamente, por ejemplo utilizando generacion de puntos aleatorios y seleccion del minimo. 

Forma analítica:

$$ RSS(\theta) = \frac{1}{2m} \sum_{i=1}^m (y^{(i)}
- x^{(i)}\theta)^2$$
m indica el número de puntos, los que son de la forma $(x^{(1)},y^{(1)}),(x^{(2)},y^{(2)})...,(x^{(m)},y^{(m)})$.
Utilizando notación matricial, RSS() queda como:

$$ RSS(\theta) = \frac{1}{2m}(Y - X\theta)^T(Y - X\theta)$$

Cabe destacar que en este ejercicio en particular, $\theta$ posee una dimensión igual a 3, Y tiene una dimensión igual a $N$ y X una dimensión de $N x 3$.

Finalmente, para encontrar los coeficientes que componen a vector $\theta$ se debe derivar RSS() con respecto a $\theta$ y luego igualar a 0, con lo que se obtiene lo siguiente.
$$\hat{\theta} = (X^T X)^{-1}X^T Y$$

Otra forma de hacer esta busqueda del minimo de RSS o funcion de costo en el lenguage de Machine Learning es utilizando una tecnica llamada Gradient Descent que es tipica en ML y que aparece explicado en el segundo item del curso Machine Learning de Andrew Ng.

In [ ]:
X_train=[]
Y_train=[]

#Los valores de SET[i][1] y SET[i][2] se colocarán en X y se vizualizarán como las nuevas variables x1 y x2.
for i in range(len(SET_train)):  
    X_train.append([1,SET_train[i][1],SET_train[i][2]])


#Ahora se colocarán los valores SET[i][3] en Y.
for i in range(len(SET_train)): 
    Y_train.append(SET_train[i][3])
    
X_train=np.array(X_train)                     
Y_train=np.array(Y_train)

In [ ]:
X_test=[]
Y_test=[]

for i in range(len(SET_test)):  
    X_test.append([1,SET_test[i][1],SET_test[i][2]])


#Ahora se colocarán los valores SET[i][3] en Y.
for i in range(len(SET_test)): 
    Y_test.append(SET_test[i][3])
    
X_test=np.array(X_test)                     
Y_test=np.array(Y_test)

In [ ]:
X_train.shape, X_test.shape , Y_train.shape , Y_test.shape

In [ ]:
XT=X_train.transpose()
XTdotX = np.dot(XT,X_train)
XTdotX

In [ ]:
XTdotX_inversa = np.linalg.inv(XTdotX)
XTdotX_inversa

In [ ]:
XTdotX_inversa_dotXT = np.dot(XTdotX_inversa, XT)

In [ ]:
theta_analitico = np.dot(XTdotX_inversa_dotXT,Y_train)
print(theta_analitico)

In [ ]:
def Costo_lineal(theta,X,Y):    
    m = len(X)
    theta = np.array(theta)
    Xtheta = np.dot(X, theta)
    YXT =(Y-Xtheta).transpose()
    YX = (Y-Xtheta)
    RSS= 1/(2*m)*np.dot(YXT,YX) 
    return RSS

In [ ]:
Costo_lineal(theta_analitico,X_train,Y_train)   

### Visualizacion de las predicciones considerando los datos del conjunto de testeo y del conjunto de entrenamiento

Con el valor del vector theta_analitico es posible visualizar como funciona este modelo de prediccion con la función lineal para los puntos del conjunto de entrenamuiento y del conjunto de testeo separadamente.

Para esto, primero se necesita crear las siguientes funciones:

In [ ]:
def funcion_lineal(x1, x2, theta): 
    
    y_prediccion = theta[0] + x1*theta[1] + x2*theta[2]
    return y_prediccion 

In [ ]:
def listas_de_predicciones_lineal(SET):

    lista_de_predicciones_azul = []
    lista_de_predicciones_rojo = []

    for pto in range(len(SET)):
        prediccion = funcion_lineal(SET[pto][1], SET[pto][2], theta_analitico)

    #Utilizando el valor de la prediccion separamos entre puntos azules y rojos
    
        if (prediccion <= 0.5):
            lista_de_predicciones_azul.append([SET[pto][1], SET[pto][2]])
        if (prediccion > 0.5):
            lista_de_predicciones_rojo.append([SET[pto][1], SET[pto][2]])
        
    return np.array(lista_de_predicciones_azul), np.array(lista_de_predicciones_rojo)

Para visualizar las predicciones hechas haremos lo siguiente.

In [ ]:
prediccion_azul_train_lineal, prediccion_rojo_train_lineal = listas_de_predicciones_lineal(SET_train)

prediccion_azul_train_lineal_t = prediccion_azul_train_lineal.transpose()
prediccion_rojo_train_lineal_t = prediccion_rojo_train_lineal.transpose()

In [ ]:
prediccion_azul_test_lineal, prediccion_rojo_test_lineal = listas_de_predicciones_lineal(SET_test)

prediccion_azul_test_lineal_t = prediccion_azul_test_lineal.transpose()
prediccion_rojo_test_lineal_t = prediccion_rojo_test_lineal.transpose()

Con esto podemos comparar visualmente las predicciones sobre el conjunto de entrenamiento y el coonjunto de testeo. A continuación se observan los 200 puntos simulados inicialmente que pertenecen al conjunto de entrenamiento.

In [ ]:
visualize_colors(ptos_observados_azul_train, ptos_observados_rojo_train)

A continuación se observa la predicción hecha por la función lineal en el conjunto de entrenamiento:

In [ ]:
visualize_colors(prediccion_azul_train_lineal_t, prediccion_rojo_train_lineal_t)

A continuación observamos los 10000 puntos creados anteriormente que pertenecen al SET_test:

In [ ]:
visualize_colors(ptos_observados_azul_test, ptos_observados_rojo_test)

A continuación se observa la predicción hecha por la función lineal para estos puntos pertenecientes a SET_test:

In [ ]:
visualize_colors(prediccion_azul_test_lineal_t, prediccion_rojo_test_lineal_t)

### Evaluación de la hipotesis lineal en terminos de las metricas "Accuracy" y el "Test Error"

Por definicion vamos a considerar que la accuracy y el test error estan dados por

$$Accuracy = \frac{PC}{PC+ PI}$$ 
Con PC los puntos predichos correctamente y PI los puntos predichos incorrectamente

$$Test_{error} = 1 - accuracy$$

Estas son métricas que nos permiten evaluar la performance de la hipótesis (en este caso la función lineal) una vez que hemos encontrado los valores de los parametros libres ($\theta$). Evaluaremos estas dos métricas en ambos conjuntos de datos (el de entrenamiento y el de testeo).

El valor del Accuracy (o la exactitud) medido en el conjunto de entrenamiento nos indica cuantitativamente el nivel de aprendizaje del modelo, dado que en este paso estamos intentando ajustar el modelo a los datos conocidos. Mientras que el Acurracy medido en el conjunto de testeo nos indica su poder de prediccion, dado que estos datos no fueron utilizados para el entrenamiento del modelo. 

Normalmente se espera que el valor de la Accuracy sea mejor (o más cercano a 1) en el conjunto de entrenamiento que en el conjunto de testeo, porque los datos del primer conjunto fueron los utilizados para calcular los parametros del modelo. Sin embargo esto no está garantizado matemáticamente.

En general es recomendable obtener un valor del accuracy cercano a 1 en el conjunto de testeo dado que nuestro objectivo final es tener poder de prediccion, no solo de ajustar los datos conocidos.

In [ ]:
def accuracy_funcion_lineal(SET, theta):

    ptos_correctos = 0
    ptos_incorrectos = 0
    
    for pto in range(len(SET)):
        prediccion = funcion_lineal(SET[pto][1], SET[pto][2], theta)
        
        if (((prediccion < 0.5) and (SET[pto][3]==0)) or
            ((prediccion > 0.5) and (SET[pto][3]==1))):
            
            ptos_correctos = ptos_correctos + 1
        else:
            ptos_incorrectos = ptos_incorrectos + 1
            
    ratio = ptos_correctos/(ptos_correctos + ptos_incorrectos)
        
        
    print("puntos correctos: %d y puntos incorrectos: %d"%(ptos_correctos, ptos_incorrectos))
        
    return ratio

### Evaluación del modelo usando el conjunto de entrenamiento

In [ ]:
train_accuracy_lineal = accuracy_funcion_lineal(SET_train, theta_analitico)
train_error_lineal = 1 - train_accuracy_lineal
print("train accuracy lineal: %.2f and test error lineal: %.2f "%(train_accuracy_lineal, train_error_lineal))

### Evaluación del modelo usando el conjunto de testeo

In [ ]:
test_accuracy_lineal = accuracy_funcion_lineal(SET_test, theta_analitico)
test_error_lineal = 1 - test_accuracy_lineal
print("test accuracy lineal: %.2f and test error lineal: %.2f "%(test_accuracy_lineal, test_error_lineal))

## Caso 2: Función Sigmoid

- Hipótesis 2: Función sigmoid ($\sigma$)
- Función de costo: $J(\theta)$ regresión logística 
- Minimización: theta obtenido por gradient descent
- Evaluación: accuracy y test error

Para este caso asumimos que el modelo de clasificación o hipotesis esta dado por una funcion del tipo 

$$ h(x_1^{(i)},x_2^{(i)}) = \sigma(\theta_0 + \theta_1 x_1^{(i)} + \theta_2 x_2^{(i)})$$
$$h(x_1^{(i)},x_2^{(i)}) = \frac{1}{1+e^{-(\theta_0 + \theta_1 x_1^{(i)} + \theta_2 x_2^{(i)})}}$$

Los valores de $\theta$ son obtenidos a partir de la busqueda del minimo de la funcion $J(\theta)$ con respecto a los datos del conjunto de entrenamiento.

$$ J(\theta) = \frac{-1}{m}\sum_{i=1}^m (y^{(i)}\log(h(x_1^{(i)},x_2^{(i)}))+(1-y^{(i)})\log(1-h(x_1^{(i)},x_2^{(i)})))$$

Este minimo puede ser encontrado analíticamente, aleatoriamente o con gradient descent. En este caso utilizaremos el método de gradient descent. Este método es un algoritmo de optimización que permite converger hacia el valor mínimo de la función de costo mediante un proceso iterativo.

$$ \theta_j := \theta_j - \alpha \frac{\partial J(\theta)}{\partial \theta_j}$$

En donde el símbolo $":="$ significa que lo que aparece al costado izquierdo de la ecuación se actualizará en cada iteración con el valor obtenido en el costado derecho de esta. El índice de aprendizaje es representado por $\alpha$, el cual indica el tamaño de los pasos que se darán en cada iteración y debe ser escogido estratégicamente. La implementación de este algoritmo se puede vectorizar como se muestra a continuación.

$$ \theta_j := \theta_j - \frac{\alpha}{m}X^T (g(X \theta)-y)) $$

### Gradient Descent

In [ ]:
X_train.shape, Y_train.shape 

In [ ]:
theta_inicial = [] 
for i in range(len(theta_analitico)):
    np.random.seed(10)
    a = np.random.uniform(theta_analitico[i] - 0.5 , theta_analitico[i] + 0.5) 
    theta_inicial.append(a)

print(theta_analitico, theta_inicial)

In [ ]:
def Sigmoid(v):
    largo = len(v)
    g=[]
    
    for i in range(largo):
        g.append(1/(1+np.exp(-v[i])))
    return np.array(g)



def Costo_Sigmoid(X, y, theta):
    Sum = 0
    m = X.shape[0]
    theta = np.array(theta)
    
    for i in range(m):
        Xtheta = np.dot(X, theta)
        Xtheta = np.array(Xtheta)
        h = Sigmoid(Xtheta)
        Sum = Sum + ((y[i]*np.log(h[i])) + (1 - y[i])*np.log(1 - h[i]))
        
    return -(Sum/m)



def Gradient_Descent(theta,X,y,alpha, iteraciones):
    X=np.array(X)
    XT=X.transpose()
    
    m = X.shape[0]
    
    lista_J=[]
    lista_iteracion = []
    
    for i in range(iteraciones):
        
        Xtheta = np.dot(X, theta)
        resta = Sigmoid(Xtheta)-y
        XTSigmoid = np.dot(XT,resta)
        
        theta = theta - (alpha/m) * XTSigmoid
        
        J = Costo_Sigmoid(X, y, theta) 
        
        lista_J.append(J)
        lista_iteracion.append(i)
    
    return np.array(lista_J) , np.array(lista_iteracion),theta

Probemos con valores de $\alpha$ chicos y 50 iteraciones:

In [ ]:
J_1,iteraciones_1,theta_1 = Gradient_Descent(theta_inicial,X_train,Y_train,0.01,50)
J_2,iteraciones_2,theta_2 = Gradient_Descent(theta_inicial,X_train,Y_train,0.1,50)
J_3,iteraciones_3,theta_3 = Gradient_Descent(theta_inicial,X_train,Y_train,1,50)
J_4,iteraciones_4,theta_4 = Gradient_Descent(theta_inicial,X_train,Y_train,5,50)

In [ ]:
plt.plot(iteraciones_1,J_1,"ro", label="alpha=0.01")
plt.plot(iteraciones_2,J_2,"bo",label="alpha=0.1")
plt.plot(iteraciones_3,J_3,"co",label="alpha=1")
plt.plot(iteraciones_4,J_4,"yo",label="alpha=5")

plt.title("Evolución función de costo")
plt.legend(loc='center right')
plt.ylabel("Costo")
plt.xlabel("Iteraciones")
plt.show()

Podemos ver que con  𝛼=1  la función de costo pudo converger rápidamente a un valor estable y cercano al mínimo. Con 50 iteraciones y con  𝛼=1 , la función de costo vale 0.4174683. Ahora veamos que sucede con valores de $\alpha$ más grandes: 

In [ ]:
J_5,iteraciones_5,theta_5 = Gradient_Descent(theta_inicial,X_train,Y_train,5,20)
J_6,iteraciones_6,theta_6 = Gradient_Descent(theta_inicial,X_train,Y_train,10,20)
J_7,iteraciones_7,theta_7 = Gradient_Descent(theta_inicial,X_train,Y_train,15,20)

In [ ]:
plt.plot(iteraciones_5,J_5,"yo", label="alpha=5")
plt.plot(iteraciones_6,J_6,"go",label="alpha=10")
plt.plot(iteraciones_7,J_7,"mo",label="alpha=15")
plt.legend(loc='center right')
plt.title("Evolución función de costo")
plt.ylabel("Costo")
plt.xlabel("Iteraciones")
plt.show()

Para resolver este notebook me quedaré con $\alpha = 7$ y con 20 iteraciones, ya que con estas variables la función de costo presenta un comportamiento estable y un valor muy pequeño. Con 20 iteraciones y con $\alpha=7$, el menor valor de la función de costo es 0.33182636 y el theta correspondiente se muestra a continuación:

In [ ]:
theta_4

In [ ]:
J_4[49]

In [ ]:
def listas_de_predicciones_sigmoid(SET,theta):

    lista_predicciones_azul = []
    lista_predicciones_rojo = []
    
    X = []
    for i in range(len(SET)):  
        X.append([1,SET[i][1],SET[i][2]])
    
    Xtheta = np.dot(X,theta)
    
    prediccion = Sigmoid(Xtheta)
    
    for pto in range(len(SET)):
    
        if (prediccion[pto] <= 0.5):
            lista_predicciones_azul.append([SET[pto][1], SET[pto][2]])
        if (prediccion[pto] > 0.5):
            lista_predicciones_rojo.append([SET[pto][1], SET[pto][2]])
        
    return np.array(lista_predicciones_azul), np.array(lista_predicciones_rojo)

### Visualización

In [ ]:
prediccion_azul_train_sigmoid, prediccion_rojo_train_sigmoid = listas_de_predicciones_sigmoid(SET_train,theta_4)

prediccion_azul_train_sigmoid_t = prediccion_azul_train_sigmoid.transpose()
prediccion_rojo_train_sigmoid_t = prediccion_rojo_train_sigmoid.transpose()

In [ ]:
prediccion_azul_test_sigmoid, prediccion_rojo_test_sigmoid = listas_de_predicciones_sigmoid(SET_test,theta_4)

prediccion_azul_test_sigmoid_t = prediccion_azul_test_sigmoid.transpose()
prediccion_rojo_test_sigmoid_t = prediccion_rojo_test_sigmoid.transpose()

Con esto podemos comparar visualmente las predicciones sobre el conjunto de entrenamiento y el conjunto de testeo, utilizando un modelo predictivo con la función sigmoid. A continuación se observan los 200 puntos simulados inicialmente que pertenecen al conjunto de entrenamiento.

In [ ]:
visualize_colors(ptos_observados_azul_train, ptos_observados_rojo_train)

A continuación se observa la predicción hecha por la función sigmoid en el conjunto de entrenamiento:

In [ ]:
visualize_colors(prediccion_azul_train_sigmoid_t, prediccion_rojo_train_sigmoid_t)

A continuación observamos los 10000 puntos creados anteriormente que pertenecen al SET_test:

In [ ]:
visualize_colors(ptos_observados_azul_test, ptos_observados_rojo_test)

A continuación se observa la predicción hecha por la función sigmoid para estos puntos pertenecientes a SET_test:

In [ ]:
visualize_colors(prediccion_azul_test_sigmoid_t, prediccion_rojo_test_sigmoid_t)

### Evaluación de la hipotesis sigmoid en terminos de las metricas "Accuracy" y el "Test Error"

Recordemos que:

$$Accuracy = \frac{PC}{PC+ PI}$$ 
Con PC los puntos predichos correctamente y PI los puntos predichos incorrectamente

$$Test_{error} = 1 - accuracy$$

Con estas métricas evaluaremos la performance de la hipótesis de la función sigmoid. Evaluaremos estas dos métricas en ambos conjuntos de datos (el de entrenamiento y el de testeo).

In [ ]:
def accuracy_funcion_sigmoid(SET, theta):

    ptos_correctos = 0
    ptos_incorrectos = 0
    
    X = []
    for i in range(len(SET)):  
        X.append([1,SET[i][1],SET[i][2]])
    
    Xtheta = np.dot(X,theta)
    
    prediccion = Sigmoid(Xtheta)
    
    for pto in range(len(SET)):
        
        if (((prediccion[pto] <= 0.5) and (SET[pto][3]==0)) or
            ((prediccion[pto] > 0.5) and (SET[pto][3]==1))):
            
            ptos_correctos = ptos_correctos + 1
        else:
            ptos_incorrectos = ptos_incorrectos + 1
            
    ratio = ptos_correctos/(ptos_correctos + ptos_incorrectos)
        
        
    print("puntos correctos: %d y puntos incorrectos: %d"%(ptos_correctos, ptos_incorrectos))
        
    return ratio

### Evaluación del modelo usando el conjunto de entrenamiento

In [ ]:
train_accuracy_sigmoid = accuracy_funcion_sigmoid(SET_train, theta_4)
train_error_sigmoid = 1 - train_accuracy_sigmoid
print("train accuracy sigmoid: %.2f and test error sigmoid: %.2f "%(train_accuracy_sigmoid, train_error_sigmoid))

### Evaluación del modelo usando el conjunto de testeo

In [ ]:
test_accuracy_sigmoid = accuracy_funcion_sigmoid(SET_test, theta_4)
test_error_sigmoid = 1 - test_accuracy_sigmoid
print("test accuracy sigmoid: %.2f and test error sigmoid: %.2f "%(test_accuracy_sigmoid, test_error_sigmoid))

## Caso 3: Redes Neuronales

- Hipótesis: Función sigmoid ($\sigma$)
- Función de costo: binary_crossentropy
- Minimización: backpropagation
- Evaluación: accuracy 

Añadiremos una capa a la vez, las que seguirán las siguientes características:

El modelo esperará filas de los datos con 2 variables, por lo que el argumento input_dim=2.
La primera capa oculta tendrá 2 nodos o neuronas (al parecer el bias no se considera), lo que obtenemos al escribir un 2 en la primera variable de entrada de Dense y tendrá una función de activación sigmoid, ya que de esta forma lo resolvimos en el resumen.
La capa de salida tendrá solo una neurona y también usará una función de activación sigmoid.


In [ ]:
#Definición del modelo utilizando Keras
model = Sequential()
model.add(Dense(2, input_dim=2, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compilación del modelo:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Visualización de la red neuronal
model.summary()

In [ ]:
# ajustar el modelo al conjunto de datos de entrenamiento (con esto se aprenden los wights)
#no debemos considerar el bias, por eso ponemos X_train[:,1:]

model.fit(X_train[:,1:], Y_train, epochs=150, batch_size=50)  

##### Evaluación del modelo usando el conjunto de entrenamiento

In [ ]:
J_train, accuracy_train = model.evaluate(X_train[:,1:], Y_train)
print('Accuracy train: %.2f' % (accuracy_train*100))

### Evaluación del modelo usando el conjunto de testeo

In [ ]:
# evaluación del modelo para el conjunto de testeo
J_test, accuracy_test = model.evaluate(X_test[:,1:], Y_test)
print('Accuracy test: %.2f' % (accuracy_test*100))

In [ ]:
prediccion_train = model.predict(X_train[:,1:]) #sin aproximar
prediccion_final_train = [round(x[0]) for x in prediccion_train] #aproxima a 1 o a 0


prediccion_test = model.predict(X_test[:,1:]) #sin aproximar
prediccion_final_test = [round(x[0]) for x in prediccion_test] #aproxima a 1 o a 0

In [ ]:
puntos_azul_train = []
puntos_rojo_train = []

for i in range(len(prediccion_final_train)):
    if prediccion_final_train[i] == 1:
        puntos_rojo_train.append(X_train[i,1:])
    else:
        puntos_azul_train.append(X_train[i,1:])
        
puntos_azul_test = []
puntos_rojo_test = []

for i in range(len(prediccion_final_test)):
    if prediccion_final_test[i] == 1:
        puntos_rojo_test.append(X_test[i,1:])
    else:
        puntos_azul_test.append(X_test[i,1:])
        

In [ ]:
puntos_azul_train_t = np.array(puntos_azul_train).transpose()
puntos_rojo_train_t = np.array(puntos_rojo_train).transpose()

puntos_azul_test_t = np.array(puntos_azul_test).transpose()
puntos_rojo_test_t = np.array(puntos_rojo_test).transpose()

In [ ]:
def visualize_colors(ptos_azul, ptos_rojo):

    x1azul = ptos_azul[0]
    x2azul = ptos_azul[1]
    
    x1rojo = ptos_rojo[0]
    x2rojo = ptos_rojo[1]
    
    plt.plot(x1azul,x2azul,"bo")
    plt.plot(x1rojo,x2rojo,"ro")
    plt.axis([-4,4,-4,4])
    plt.title("Distribucion de puntos segun el color")
    plt.show() 

In [ ]:
visualize_colors(puntos_azul_train_t, puntos_rojo_train_t)

In [ ]:
visualize_colors(puntos_azul_test_t, puntos_rojo_test_t)

## Conclusiones

Comparando el valor del Acurracy obtenido para los tres modelos analizados, podemos notar que estos presentan un valor de Acurracy bastante similar. Esto muestra la consistencia del método gradient descent para encontrar los parámetros libres de J($\theta$), ya que obtuvo el mismo error que se calculó anteriormente de manera analítica para la hipótesis lineal. Para aumentar la presición de la red neuronal se podrían utilizar más neuronas en la capa oculta o agregar más capas ocultas.